<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Aerospike-Queries-in-Python¶" data-toc-modified-id="Aerospike-Queries-in-Python¶-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Aerospike Queries in Python¶</a></span><ul class="toc-item"><li><span><a href="#Ensure-database-is-running" data-toc-modified-id="Ensure-database-is-running-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Ensure database is running</a></span></li><li><span><a href="#Connect-to-database-and-populate-test-data" data-toc-modified-id="Connect-to-database-and-populate-test-data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Connect to database and populate test data</a></span></li><li><span><a href="#Create-secondary-index" data-toc-modified-id="Create-secondary-index-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Create secondary index</a></span></li></ul></li><li><span><a href="#Querying-with-secondary-indexes" data-toc-modified-id="Querying-with-secondary-indexes-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Querying with secondary indexes</a></span><ul class="toc-item"><li><span><a href="#Create-a-query" data-toc-modified-id="Create-a-query-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Create a query</a></span></li><li><span><a href="#Project-bins" data-toc-modified-id="Project-bins-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Project bins</a></span></li><li><span><a href="#Add-query-predicate" data-toc-modified-id="Add-query-predicate-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Add query predicate</a></span></li><li><span><a href="#Define-foreach-function" data-toc-modified-id="Define-foreach-function-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Define foreach function</a></span></li><li><span><a href="#Execute-query-and-foreach" data-toc-modified-id="Execute-query-and-foreach-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Execute query and foreach</a></span></li><li><span><a href="#Explore-other-query-capabilities" data-toc-modified-id="Explore-other-query-capabilities-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Explore other query capabilities</a></span></li><li><span><a href="#Clean-up" data-toc-modified-id="Clean-up-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Clean up</a></span></li><li><span><a href="#Next-steps" data-toc-modified-id="Next-steps-2.8"><span class="toc-item-num">2.8&nbsp;&nbsp;</span>Next steps</a></span></li></ul></li></ul></div>

# Aerospike Queries in Python
Intoduction to Aerospike queries in Python.
<br>
This notebook requires Aerospike datbase running on localhost and that python and the Aerospike python client have been installed (`pip install aerospike`). Visit [Aerospike notebooks repo](https://github.com/aerospike-examples/interactive-notebooks) for additional details and the docker container.

## Ensure database is running
This notebook requires that Aerospike datbase is running.

In [1]:
!asd >& /dev/null
!pgrep -x asd >/dev/null && echo "Aerospike database is running!" || echo "**Aerospike database is not running!**"

Aerospike database is running!


## Connect to database and populate test data
The test data has ten records with user-key "id1-10", two bins (fields) "name" and "age", in the namespace "test" and set "demo". 

In [2]:
# import the module
from __future__ import print_function
import aerospike

# Configure the client
config = {
  'hosts': [ ('127.0.0.1', 3000) ],
  'policy' : {'key': aerospike.POLICY_KEY_SEND}
}

# Create a client and connect it to the cluster
try:
  client = aerospike.client(config).connect()
except:
  import sys
  print("failed to connect to the cluster with", config['hosts'])
  sys.exit(1)

# Records are addressable via a tuple of (namespace, set, key)
people = [ {'id':1, 'name':'John Doe', 'age': 53},
           {'id':2, 'name':'Brian Yu', 'age': 21},
           {'id':3, 'name':'Will Kim', 'age': 34},
           {'id':4, 'name':'Dorothy Smith', 'age': 48},
           {'id':5, 'name':'Sara Poe', 'age': 29},
           {'id':6, 'name':'Kim Knott', 'age': 56},
           {'id':7, 'name':'Joe Miller', 'age': 30},
           {'id':8, 'name':'Jeff Nye', 'age': 32},
           {'id':9, 'name':'Jane Doe', 'age': 44},
           {'id':10, 'name':'Emily Tuck', 'age': 22} ]
try:
    for i in range(10):
      # Write the records
      client.put(('test', 'demo', 'id'+str(people[i]['id'])), people[i])
except Exception as e:
  import sys
  print("error: {0}".format(e), file=sys.stderr)

print('Test data populated.')

Test data populated.


## Create secondary index
To use the query API, a secondary index must exist on the query field. We will create an integer secondary index on the "age" bin.

In [3]:
# Must create an index to query on a bin
from aerospike import exception as ex
try:
    client.index_integer_create("test", "demo", "age", "test_demo_number_idx")
except ex.IndexFoundError:
    pass

print('Secondary index created.')

Secondary index created.


# Querying with secondary indexes

In addition to retrieving records with the primary index using the key-value store APIs, the Aerospike Python client provides an API to query records using secondary indexes. To use the query API, a secondary index must exist on the query field.

Use the Query APIs to query the database using secondary indexes.

## Create a query
The API client.query() takes the namespace (required) and set (optional) arguments. The parameter set can be omitted or None, in which case records in the namespace that are outside any set are returned. The return value is a new aerospike.Query class instance.

This example creates a query on the test namespace, demo set.

In [4]:
query = client.query('test', 'demo')
print('Query object created.')

Query object created.


## Project bins
Project (or select) bins using select() on the Query class instance. The select() API accepts one or many bin names (strings).

This example selects "name" and "age" bins from the specified records.

In [5]:
query.select('name', 'age')
print('Bins name and age selected.')

Bins name and age selected.


## Add query predicate
Define predicates using the where() API on the Query class instance. The where() API accepts a predicate created using one of the functions in aerospike.predicates including:

- equals(bin, value) — Find records containing the bin with the specified value (integer or string).
- between(bin, min, max) — Find records containing the bin with a value in the min and max range (integer only).

This example adds the between() predicate to a query.

In [6]:
from aerospike import predicates as p
query.where( p.between('age', 14, 25) )
print('Predicate defined.')

Predicate defined.


## Define foreach function
In order to executer the query and read the results, we need to use the foreach() API in the Query class instance. The foreach() API accepts a callback function for each record read from the query. The callback function must accept a single argument as a tuple:

- key tuple — The tuple to identify the record.
- metadata — The dict containing the record metadata (TTL and generation).
- record — The dict containing the record bins.

If the callback returns False, the client stops reading results.

This examples executes the query and prints results as they are read.

To print the records as they are read, we define a print_result function.

In [7]:
def print_result(result_tuple):
    print(result_tuple)
    
print('Foreach function defined.')

Foreach function defined.


## Execute query and foreach
Now we are ready to execute the query by passing in the print_result that will be called for each record. Based on the data we populated earlier, we expect 2 results between ages 14 and 25.

In [8]:
print("Executing query and printing results:")
query.foreach(print_result)

Executing query and printing results:
(('test', 'demo', None, bytearray(b'\xb2\x13X\x1dI\xd8\xba`\xab\x96\xa2\xf0\xd9\x8b\x19\xf9DZug')), {'ttl': 2591998, 'gen': 1}, {'name': 'Brian Yu', 'age': 21})
(('test', 'demo', None, bytearray(b'\x0bR\xbc\xa1\x02`SF?\x01\xe7\xd3`\x8d[F\xcb\xd71V')), {'ttl': 2591998, 'gen': 1}, {'name': 'Emily Tuck', 'age': 22})


## Explore other query capabilities
Please feel free to play with the "equals" predicate, adding secondary indexes on other fields, populating more test data to the "null" set and querying those records, and so on.

## Clean up

In [9]:
# Close the connection to the Aerospike cluster
client.close()
print('Connection closed.')

Connection closed.


## Next steps

Visit [Aerospike notebooks repo](https://github.com/aerospike-examples/interactive-notebooks) to run additional Aerospike notebooks. To run a different notebook, download the notebook from the repo to your local machine, and then click on File->Open, and select Upload.